In [2]:
'''
    In this guide, we'll learn 
        1. how TensorFlow allows you to make simple changes to your code to get graphs, 
        2. how graphs are stored and represented, 
        3. and how you can use them to accelerate your models.      
'''

"\n    In this guide, we'll learn \n        1. how TensorFlow allows you to make simple changes to your code to get graphs, \n        2. how graphs are stored and represented, \n        3. and how you can use them to accelerate your models.      \n"

#### **TF.function**
> * **`tf.function`: allows us to switch from eager-function to graph function**
> * **`EagerExecution`: TensorFlow operations are executed by Python, operation by operation, and returning results back to Python**
> * **`GraphExecution:` Graph execution means that tensor computations are executed as a TensorFlow graph**

#### **TF.graph**
> * **Graphs are data structures that contain a set of `tf.Operation` objects, which represent units of computation and `tf.Tensor` objects, which represent the units of data that flow between operations.**
> * **Graphs increases Model `portability` outside of Python and increases `performance`.**
> * **Portability: environments without Python interpreter, eg mobile apps, embedded devices, and backend servers**
> * **TensorFlow uses graphs as the `format for saved models` when it exports them from Python.**

##### **`Graph Optimization`:**
> * **`Constant Folding`: Statically infer the value of tensors by folding constant nodes in computation.**
> * **Separate sub-parts of a computation that are independent and split them between `threads or devices`.**
> * **Simplify arithmetic operations by eliminating common sub-expressions.**

**There is an entire optimization system, `Grappler`, to perform this and other speedups.**

### **Setup**

In [3]:
import tensorflow as tf
import timeit as tit
from datetime import datetime

### **Taking advantage of Graphs**

In [4]:
# Defining a regular python function
def func(x, y, b):
    ten = tf.matmul(x, y)
    ten = ten + b
    return ten

# convert regular function to tf.graph
graph_fn = tf.function(func)


# Creating test tensors
t1 = tf.constant([[1., 4., 5.5]]) # 1 x 3
t2 = tf.random.uniform(shape=(3, 2), minval=10, maxval=50, dtype=tf.float32, seed=25) # 3 x 3
t3 = tf.random.normal(shape=(1,2)) # 1 x 3 
print('Printing Tensor T1\n', t1.numpy())
print()
print('Printing Tensor T2\n', t2.numpy())
print()
print('Printing Tensor T3\n', t3.numpy())
print()

regular_fn_result = func(t1,t2,t3)
print('Result of Python function: ', regular_fn_result.numpy(), '\n')

graph_fn_result = graph_fn(t1, t2, t3)
print('Result of TF.Graph function: ', graph_fn_result.numpy(), '\n')

# Checking equility of the results
try:
    assert(regular_fn_result == graph_fn_result)
except:
    print('The function failed')


Printing Tensor T1
 [[1.  4.  5.5]]

Printing Tensor T2
 [[33.822636 10.392761]
 [26.041779 32.228683]
 [20.323357 30.947838]]

Printing Tensor T3
 [[-1.4044924   0.42913023]]

Result of Python function:  [[248.36371 309.94974]] 

Result of TF.Graph function:  [[248.36371 309.94974]] 

The function failed


In [5]:

def inner_fn(t1,t2,t3):
    ten = tf.matmul(t1,t2)
    ten = ten * t3
    return ten
# Using @tf.function directly to create graphs
@tf.function
def outr_fn(x):
    t1 = tf.constant([[3.],[6.]])
    t2 = tf.constant(4.5)

    # tf.function applies to a function and all other functions it calls
    return inner_fn(x, t1, t2)
        
# Calling the tf function
t1 = tf.ones(shape=(1, 2))
print(outr_fn(t1))


tf.Tensor([[40.5]], shape=(1, 1), dtype=float32)


* ##### **Python funtions to graphs**

In [6]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0
    
# `graph_relu` is a TensorFlow `Function` that wraps `simple_relu`
graph_relu = tf.function(simple_relu)
print(f'First Branch result: {graph_relu(tf.constant(10)).numpy()}')
print(f'Second Branch result: {graph_relu(tf.constant(-10)).numpy()}')


First Branch result: 10
Second Branch result: 0


In [7]:
# Reading the graphs code using tf.autograph.to_code()
print(tf.autograph.to_code(simple_relu))

# Easy to print, impossible to read!!

def tf__simple_relu(x):
    with ag__.FunctionScope('simple_relu', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_bo

In [8]:
# Graph generated from the `simple_relu` function
print(f'Printing the actual graph of the function')
print(graph_relu.get_concrete_function(tf.constant(2)).graph.as_graph_def())


Printing the actual graph of the function
node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "Tcond"
    value {
      type: DT_BOOL
    }
  }
  attr {
    key: "Tin"
    value {
      list {
        type: DT_INT32
      }
    }
  }
  attr {
    key: "Tout"
    value {
      list {
        type: DT_BO

* ##### **Polymorphism: One funtion, many graphs**
> * **Each time you invoke a Function with new dtypes and shapes in its arguments, Function creates a new `tf.Graph` for the new arguments.** 
> * **The dtypes and shapes of a tf.Graph's inputs are known as an `input signature`.**
> * **The Function stores the tf.Graph corresponding to that signature in a `ConcreteFunction`. A ConcreteFunction is a wrapper around a tf.Graph.**


In [9]:
@tf.function
def my_relu(x):
    print('Python code!!')
    return tf.greater(0., x)

print('In the top 2 examples, a new graph is instantaited to back the operation as a new signature is received\n')
print(my_relu(tf.constant(3.)))
print(my_relu(
    tf.constant([
        [3.], [-5.]
    ])
))
print()

print('No new graph is instantiated as the function has already seen the input signatures')
print(my_relu(tf.constant(3.)))
print(my_relu(
    tf.constant([
        [-12.], [10]
    ])
))


In the top 2 examples, a new graph is instantaited to back the operation as a new signature is received

Python code!!
tf.Tensor(False, shape=(), dtype=bool)
Python code!!
tf.Tensor(
[[False]
 [ True]], shape=(2, 1), dtype=bool)

No new graph is instantiated as the function has already seen the input signatures
tf.Tensor(False, shape=(), dtype=bool)
tf.Tensor(
[[ True]
 [False]], shape=(2, 1), dtype=bool)


In [10]:
print(my_relu.pretty_printed_concrete_signatures()) 

my_relu(x)
  Args:
    x: float32 Tensor, shape=()
  Returns:
    bool Tensor, shape=()

my_relu(x)
  Args:
    x: float32 Tensor, shape=(2, 1)
  Returns:
    bool Tensor, shape=(2, 1)


### **Using TF.function**
**`Getting Function to work properly`**

* ##### **Graph Exec vs Eager Exec**

In [32]:
# Creating a mean-squared-err func replica
@tf.function
def MSE_fn(y_tr, y_pr):
    print("Running Eagerly")
    sq_diff = tf.pow(y_tr - y_pr,2)
    return tf.reduce_mean(sq_diff)

# Declaring constants
t1 = tf.random.normal(shape= [2, 2], mean=7, stddev=2.5, dtype=tf.float32)
t2 = tf.random.uniform(shape= [2, 2], minval=5, maxval= 13, dtype=tf.float32)

print(f'Tensor 1:\n {tf.cast(t1, tf.int32)}')
print(f'Tensor 2:\n {tf.cast(t2, tf.int32)}\n')

ten = MSE_fn(t1, t2)
print('Mean-squared-error for the above tensors')
ten.numpy()


Tensor 1:
 [[2 8]
 [8 4]]
Tensor 2:
 [[11  7]
 [ 6  8]]

Running Eagerly
Mean-squared-error for the above tensors


23.934187

In [33]:
"""
    To verify that your Function's graph is doing the same computation 
    as its equivalent Python function, you can make it execute eagerly 
    with tf.config.run_functions_eagerly(True). This is a switch that 
    turns off Function's ability to create and run graphs, instead executing 
    the code normally.
"""

# Switching to eager mode
tf.config.run_functions_eagerly(True)

print(MSE_fn(t1, t2).numpy())

# Returning back to graph execution
tf.config.run_functions_eagerly(False)


Running Eagerly
23.934187


**Tracing captures the TensorFlow operations into a graph, while python code is not captured in the graph. That graph is then executed without ever running the Python code again**

In [36]:
y1 = MSE_fn(t1, t1 * 1.25)
y1

<tf.Tensor: shape=(), dtype=float32, numpy=2.8282628>

* ##### **Non-strict Execution**

In [40]:
def unused_eager(xten):
    tf.gather(xten, [1])
    return xten

t1 = tf.constant([2.3])

try:
    unused_eager(t1)

except tf.errors.InvalidArgumentError as e:
    print(f'Error: {e}')
    
non_strict_func = tf.function(unused_eager)
non_strict_func(t1).numpy()

Error: indices[0] = 1 is not in [0, 1) [Op:GatherV2]


array([2.3], dtype=float32)

* ##### **Best practices** 

### **Optimization benchmarks**

* ##### **Performance**